# `interrupt`를 사용하여 사용자 입력을 기다리는 방법

!!! tip "필수 조건"

    이 가이드는 다음 개념에 대한 익숙함을 가정합니다:

    * [Human-in-the-loop](../../../concepts/human_in_the_loop)
    * [LangGraph 용어집](../../../concepts/low_level)
    

**Human-in-the-loop (HIL)** 상호 작용은 [에이전트 시스템](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#human-in-the-loop)에 중요합니다. 사람의 입력을 기다리는 것은 일반적인 HIL 상호 작용 패턴으로, 에이전트가 사용자에게 명확화 질문을 하고 진행하기 전에 입력을 기다릴 수 있습니다.

LangGraph에서 [`interrupt()`][langgraph.types.interrupt] 함수를 사용하여 이를 구현할 수 있습니다. `interrupt`를 사용하면 그래프 실행을 중지하여 사용자로부터 입력을 수집하고 수집된 입력으로 실행을 계속할 수 있습니다.

## 설정

먼저 필요한 패키지를 설치해야 합니다

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

다음으로 Anthropic 및/또는 OpenAI(사용할 LLM)에 대한 API 키를 설정해야 합니다

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요. 
    </p>
</div>

## 간단한 사용법

사람의 피드백을 사용하는 기본 예제를 살펴보겠습니다. 간단한 접근 방식은 사용자 입력을 수집하도록 특별히 설계된 노드 **`human_feedback`**를 만드는 것입니다. 이를 통해 그래프의 특정 선택된 지점에서 피드백을 수집할 수 있습니다.

단계:

1. **`human_feedback`** 노드 내부에서 **`interrupt()`**를 호출합니다.
2. 이 노드까지 그래프의 상태를 저장하기 위해 **[체크포인터](https://langchain-ai.github.io/langgraph/concepts/low_level/#checkpointer)**를 설정합니다.
3. **`Command(resume=...)`**를 사용하여 요청된 값을 **`human_feedback`** 노드에 제공하고 실행을 재개합니다.

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

# highlight-next-line
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver
from IPython.display import Image, display


class State(TypedDict):
    input: str
    user_feedback: str


def step_1(state):
    print("---Step 1---")
    pass


def human_feedback(state):
    print("---human_feedback---")
    # highlight-next-line
    feedback = interrupt("Please provide feedback:")
    return {"user_feedback": feedback}


def step_3(state):
    print("---Step 3---")
    pass


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# 메모리 설정
memory = InMemorySaver()

# 추가
graph = builder.compile(checkpointer=memory)

# 보기
display(Image(graph.get_graph().draw_mermaid_png()))

`human_feedback`의 `interrupt()`까지 실행합니다:

In [ ]:
# 입력
initial_input = {"input": "hello world"}

# 스레드
thread = {"configurable": {"thread_id": "1"}}

# 첫 번째 중단까지 그래프 실행
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

이제 사용자 입력으로 그래프 상태를 수동으로 업데이트할 수 있습니다:

In [ ]:
# 그래프 실행 계속
for event in graph.stream(
    # highlight-next-line
    Command(resume="go to step 3!"),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

피드백이 상태에 추가된 것을 볼 수 있습니다 - 

In [6]:
graph.get_state(thread).values

{'input': 'hello world', 'user_feedback': 'go to step 3!'}

## 에이전트

[에이전트](../../../concepts/agentic_concepts) 컨텍스트에서 사용자 피드백을 기다리는 것은 명확화 질문을 하는 데 특히 유용합니다. 이를 설명하기 위해 [도구 호출](https://python.langchain.com/docs/concepts/tool_calling/)이 가능한 간단한 [ReAct 스타일 에이전트](../../../concepts/agentic_concepts#react-implementation)를 만들 것입니다.

이 예제에서는 Anthropic의 채팅 모델과 **모의 도구**(순전히 시연 목적)를 사용합니다.

<div class="admonition note">
    <p class="admonition-title">LangChain과 함께 Pydantic 사용하기</p>
    <p>
        이 노트북은 Pydantic v2 <code>BaseModel</code>을 사용하며, <code>langchain-core >= 0.3</code>이 필요합니다. <code>langchain-core < 0.3</code>을 사용하면 Pydantic v1 및 v2 <code>BaseModels</code>의 혼합으로 인해 오류가 발생합니다.
    </p>
</div>  

In [ ]:
# 상태 설정
from langgraph.graph import MessagesState, START

# 도구 설정
# 하나의 실제 도구 - 검색 도구가 있습니다
# 또한 하나의 "가짜" 도구 - "ask_human" 도구도 있습니다
# 여기서 실제 도구를 정의합니다
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode


@tool
def search(query: str):
    """웹을 서핑하기 위해 호출합니다."""
    # 이것은 실제 구현을 위한 플레이스홀더입니다
    # 하지만 LLM에게는 알리지 마세요 😊
    return f"I looked up: {query}. Result: It's sunny in San Francisco, but you better look out if you're a Gemini 😈."


tools = [search]
tool_node = ToolNode(tools)

# 모델 설정
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

from pydantic import BaseModel


# 모든 도구를 모델에 "바인딩"할 것입니다
# 위의 실제 도구가 있지만 사람에게 묻는 모의 도구도 필요합니다
# `bind_tools`가 도구뿐만 아니라 도구 정의도 받으므로,
# `ask_human`에 대한 도구 정의를 정의할 수 있습니다
class AskHuman(BaseModel):
    """사람에게 질문하기"""

    question: str


model = model.bind_tools(tools + [AskHuman])

# 노드 및 조건부 엣지 정의


# 계속할지 여부를 결정하는 함수 정의
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # 함수 호출이 없으면 종료합니다
    if not last_message.tool_calls:
        return END
    # 도구 호출이 Human에게 묻는 것이면 해당 노드를 반환합니다
    # 여기에 일부 시스템에 Human 입력이 필요한 것이 있음을 알리는 로직을 추가할 수도 있습니다
    # 예를 들어 슬랙 메시지 전송 등
    elif last_message.tool_calls[0]["name"] == "AskHuman":
        return "ask_human"
    # 그렇지 않으면 계속합니다
    else:
        return "action"


# 모델을 호출하는 함수 정의
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # 기존 목록에 추가될 것이므로 리스트를 반환합니다
    return {"messages": [response]}


# 사람에게 묻는 가짜 노드 정의
def ask_human(state):
    tool_call_id = state["messages"][-1].tool_calls[0]["id"]
    ask = AskHuman.model_validate(state["messages"][-1].tool_calls[0]["args"])
    # highlight-next-line
    location = interrupt(ask.question)
    tool_message = [{"tool_call_id": tool_call_id, "type": "tool", "content": location}]
    return {"messages": tool_message}


# 그래프 빌드

from langgraph.graph import END, StateGraph

# 새 그래프 정의
workflow = StateGraph(MessagesState)

# 순환할 세 개의 노드 정의
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)
workflow.add_node("ask_human", ask_human)

# 엔트리포인트를 `agent`로 설정
# 이것은 이 노드가 첫 번째로 호출된다는 것을 의미합니다
workflow.add_edge(START, "agent")

# 이제 조건부 엣지를 추가합니다
workflow.add_conditional_edges(
    # 먼저 시작 노드를 정의합니다. `agent`를 사용합니다.
    # 이것은 `agent` 노드가 호출된 후 취하는 엣지를 의미합니다.
    "agent",
    # 다음으로 어느 노드가 다음에 호출될지 결정하는 함수를 전달합니다.
    should_continue,
    path_map=["ask_human", "action", END],
)

# 이제 `tools`에서 `agent`로 일반 엣지를 추가합니다.
# 이것은 `tools`가 호출된 후 `agent` 노드가 다음에 호출된다는 것을 의미합니다.
workflow.add_edge("action", "agent")

# 사람의 응답을 받은 후 에이전트로 돌아갑니다
workflow.add_edge("ask_human", "agent")

# 메모리 설정
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

# 마지막으로 컴파일합니다!
# 이것은 LangChain Runnable로 컴파일됩니다,
# 즉 다른 runnable과 동일하게 사용할 수 있습니다
app = workflow.compile(checkpointer=memory)

display(Image(app.get_graph().draw_mermaid_png()))

## 에이전트와 상호 작용

이제 에이전트와 상호 작용할 수 있습니다. 사용자가 어디에 있는지 묻도록 한 다음 날씨를 알려주도록 요청해 봅시다.

이것은 먼저 `ask_human` 도구를 사용한 다음 일반 도구를 사용하도록 만들어야 합니다.

In [8]:
config = {"configurable": {"thread_id": "2"}}
for event in app.stream(
    {
        "messages": [
            (
                "user",
                "Ask the user where they are, then look up the weather there",
            )
        ]
    },
    config,
    stream_mode="values",
):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ask the user where they are, then look up the weather there
================================== Ai Message ==================================

[{'text': "I'll help you with that. Let me first ask the user about their location.", 'type': 'text'}, {'id': 'toolu_012Z9yyZjvH8xKgMShgwpQZ9', 'input': {'question': 'Where are you located?'}, 'name': 'AskHuman', 'type': 'tool_use'}]
Tool Calls:
  AskHuman (toolu_012Z9yyZjvH8xKgMShgwpQZ9)
 Call ID: toolu_012Z9yyZjvH8xKgMShgwpQZ9
  Args:
    question: Where are you located?


In [9]:
app.get_state(config).next

('ask_human',)

그래프가 `ask_human` 노드 내부에서 중단되었고 `location`이 제공되기를 기다리고 있는 것을 볼 수 있습니다. `Command(resume="<location>")` 입력으로 그래프를 호출하여 이 값을 제공할 수 있습니다:

In [10]:
for event in app.stream(
    # highlight-next-line
    Command(resume="san francisco"),
    config,
    stream_mode="values",
):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'text': "I'll help you with that. Let me first ask the user about their location.", 'type': 'text'}, {'id': 'toolu_012Z9yyZjvH8xKgMShgwpQZ9', 'input': {'question': 'Where are you located?'}, 'name': 'AskHuman', 'type': 'tool_use'}]
Tool Calls:
  AskHuman (toolu_012Z9yyZjvH8xKgMShgwpQZ9)
 Call ID: toolu_012Z9yyZjvH8xKgMShgwpQZ9
  Args:
    question: Where are you located?
================================= Tool Message =================================

san francisco
================================== Ai Message ==================================

[{'text': "Now I'll search for the weather in San Francisco.", 'type': 'text'}, {'id': 'toolu_01QrWBCDouvBuJPZa4veepLw', 'input': {'query': 'current weather in san francisco'}, 'name': 'search', 'type': 'tool_use'}]
Tool Calls:
  search (toolu_01QrWBCDouvBuJPZa4veepLw)
 Call ID: toolu_01QrWBCDouvBuJPZa4veepLw
  Args:
    query: current weather in san francisco
=